In [ ]:
import os, sys
current_path = os.getcwd()
current_path, tail = os.path.split(current_path)
folder_path= os.path.join(current_path, "libraries/")
file_path_ingest = folder_path + "Ingest.py"
file_path_calculations = folder_path + "Calculations.py"
%run $file_path_ingest
%run $file_path_calculations

In [ ]:
directory_lambertian = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Recordings\Viper Recordings\Smoke_Test_VP04\2024-07-02_15-25-50.956_measurement_VPR004_Target0.61x0.61_GT9.99_10P\MI5-WIN10"
directory_stop_sign = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Recordings\Viper Recordings\Smoke_Test_VP04\2024-07-03_13-02-30.808_measurement_Stop30_GT10.03_VPR004\RRL0283W"
directory_pixels = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Recordings\Viper Recordings\Smoke_Test_VP04\2024-07-02_10-15-36.712_measurement_pixels_VPR004\MI5-WIN10"
directory_general = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Recordings\Viper Recordings\Smoke_Test_VP04"

EXTRACT VIPER ID, RANGE GROUND TRUTH, TARGET SIZE GROUND TRUTH AND TARGET TYPE

In [ ]:
def directory_data_extractor(directory):
 indx=0
 for i in directory:
    indx=indx+1
    if directory[indx:indx+3]=='VPR' or directory[indx:indx+3]=='vpr':
        viper_id = directory[indx+4:indx+6]
        viper_id = 'VPR0'+viper_id
        print('Viper ID is:',viper_id)
        break

 indx=0
 for i in directory:
    indx=indx+1
    if directory[indx:indx+2]=='GT' or directory[indx:indx+2]=='gt':
        if float(directory[indx+2])<10 and directory[indx+4]!='.':
         ground_truth_str = directory[indx+2:indx+6]
         ground_truth = float(ground_truth_str)
         print('Ground Truth Range is:',ground_truth)
         break
        else:
         ground_truth_str = directory[indx+2:indx+7]
         ground_truth = float(ground_truth_str)
         print('Ground Truth Range is:',ground_truth)
         break

 indx=0
 for i in directory:
    indx=indx+1
    if directory[indx:indx+6]=='Target' or directory[indx:indx+6]=='target':
        
        if directory[indx+6]=='0':
            size_a=float(directory[indx+7:indx+10])
            size_b=float(directory[indx+12:indx+15])
            gt_target_size_area= size_a*size_b
            target_type = 'Lambertian'
            print('Target Size Ground Truth Area:',gt_target_size_area)
            print("Target is a type:",target_type)
            break
        else:
         gt_target_size_area = float(directory[indx+6])*float(directory[indx+8])
         target_type = 'Lambertian'
         print('Target Size Ground Truth Area:',gt_target_size_area)
         print("Target is a type:",target_type)
         break
    elif directory[indx:indx+6] == 'Stop24' or directory[indx:indx+4] == 'stop24':
        gt_target_size_area = 0.31397
        target_type = '24" Stop'
        print('Target Size Ground Truth Area:',gt_target_size_area)
        print(target_type)
        break
    elif directory[indx:indx+6] == 'Stop30' or directory[indx:indx+4] == 'stop30':
        gt_target_size_area = 0.49443
        target_type = '30" Stop'
        print('Target Size Ground Truth Area:',gt_target_size_area)
        print(target_type)
        break

 indx=0
 for i in directory:
    indx=indx+1
    if directory[indx:indx+6]=='Pixels' or directory[indx:indx+6]=='pixels':
        target_type = 'Pixels'
        print(target_type)

 indx=0
 for i in directory:
    indx=indx+1
    if directory[indx:indx+4]=='2024' or directory[indx:indx+4]=='2023'or directory[indx:indx+4]=='2025':
        test_date = directory[indx+5:indx+7]+"/"+directory[indx+8:indx+10]+"/"+directory[indx+2:indx+4]
        print("Test Date:",test_date)
        
 if target_type != 'Pixels':
  return viper_id,ground_truth,gt_target_size_area,target_type,test_date
 else:
  return viper_id,target_type,test_date


## LAMBERTIAN ANALYSIS

In [ ]:
sensor_type = "SRL" 

if os.path.exists(directory_lambertian) != True :logger.warning(f"Directory not found!, Is the drive connected?")
else:
    sensor_viper = sensor(sensor_type, directory_lambertian, debug=True)

viper_id, ground_truth, gt_target_size_area, target_type, test_date = directory_data_extractor(directory_lambertian)

DATA INGEST

In [ ]:
sensor_viper.data_read(multiple_files=False)

In [ ]:
sensor_viper.row_col_fix()

In [ ]:
sensor_viper.data_validation()
sensor_viper.df_single_frame
sensor_viper.df_single_frame = sensor_viper.df_single_frame[sensor_viper.df_single_frame['return_idx'] == 0]

TARGET SELECTION

In [ ]:
if target_type != 'Pixels':
 target_selection(sensor_viper)
 data_parting(sensor_viper)

RANGE ACCURACY / POINTS ON TARGET / PD

In [ ]:
if target_type != 'Pixels':
 pd_points = pd_calculation(sensor_viper)
 range_readout = sensor_viper.df_all_frames_target['range'].mean()
 range_accuracy= ((ground_truth - sensor_viper.df_all_frames_target['range'].mean())**2)**0.5
 print('Range Accuracy:', range_accuracy)

RANGE PRECISION

In [ ]:
if target_type != 'Pixels': 
 df_return_precision, average_std = range_precision(sensor_viper)
 average_std

TARGET SIZE

In [ ]:
if target_type != 'Pixels':
 TargHeight, TargWidth, TargPerimeter, TargArea = target_size(sensor_viper)
 factor_area = TargArea/gt_target_size_area
 print("Target is about ",TargHeight,"m"" by", TargWidth," m", "Approximate Perimeter:",TargPerimeter, "Approximate Area:",TargArea, "Target Error:", factor_area)

INTESITY DATA COLLECTION

In [ ]:
if target_type != 'Pixels':
 average_int = sensor_viper.df_all_frames_target['intensity'].mean()
 min_int = sensor_viper.df_all_frames_target['intensity'].min()
 max_int = sensor_viper.df_all_frames_target['intensity'].max()

In [ ]:
if target_type != 'Pixels':
 df_smoke_t_lambertian=pd.DataFrame({'Sample':[viper_id],
                                     'Test Date':[test_date],
                                    'Target Type':[target_type],
                                    'Ground Truth':[ground_truth],
                                    'Range Readout':[range_readout],
                                    'Range Accuracy':[range_accuracy],
                                    'Range Precision':[average_std],
                                    '90% PD Points':[pd_points],
                                    'Average Intensity':[average_int],
                                    'Min Intensity':[min_int],
                                    'Max Intensity':[max_int],
                                    'GT Target Size':[gt_target_size_area],
                                    'Target Area':[TargArea],
                                    'Mult Factor':[factor_area]
                                    })           
 
df_smoke_t_lambertian

In [ ]:
if target_type != 'Pixels':
 
 print('Range Readout:', range_readout)
 print('Range Accuracy:', range_accuracy)
 print('Range Precision:', average_std)
 print('Points 90% PD on Target:',pd_points)
 print('Average Intensity on Target:',average_int)
 print('Min Intensity on Target:',min_int)
 print('Max Intensity on Target:',max_int)
 print('Target Area:', TargArea)
 print('Multiplicative Factor', factor_area)

## STOP SIGN ANALYSIS

In [ ]:
%run $file_path_ingest
%run $file_path_calculations

sensor_type = "SRL" 

if os.path.exists(directory_stop_sign) != True :logger.warning(f"Directory not found!, Is the drive connected?")
else:
    sensor_viper = sensor(sensor_type, directory_stop_sign, debug=True)

viper_id, ground_truth, gt_target_size_area, target_type, test_date = directory_data_extractor(directory_stop_sign)

In [ ]:
sensor_viper.data_read(multiple_files=False)
sensor_viper.row_col_fix()
sensor_viper.data_validation()

sensor_viper.df_single_frame
sensor_viper.df_single_frame = sensor_viper.df_single_frame[sensor_viper.df_single_frame['return_idx'] == 0]

In [ ]:
sensor_viper.df_single_frame

In [ ]:
if target_type != 'Pixels':
 target_selection(sensor_viper)
 data_parting(sensor_viper)

In [ ]:
if target_type != 'Pixels':
 pd_points = pd_calculation(sensor_viper)
 range_readout = sensor_viper.df_all_frames_target['range'].mean()
 range_accuracy= ((ground_truth - sensor_viper.df_all_frames_target['range'].mean())**2)**0.5
 print('Range Accuracy:', range_accuracy)

In [ ]:
if target_type != 'Pixels': 
 df_return_precision, average_std = range_precision(sensor_viper)
 average_std

In [ ]:
if target_type != 'Pixels':
 TargHeight, TargWidth, TargPerimeter, TargArea = target_size(sensor_viper)
 factor_area = TargArea/gt_target_size_area
 print("Target is about ",TargHeight,"m"" by", TargWidth," m", "Approximate Perimeter:",TargPerimeter, "Approximate Area:",TargArea, "Target Error:", factor_area)

In [ ]:
if target_type != 'Pixels':
 average_int = sensor_viper.df_all_frames_target['intensity'].mean()
 min_int = sensor_viper.df_all_frames_target['intensity'].min()
 max_int = sensor_viper.df_all_frames_target['intensity'].max()

In [ ]:
if target_type != 'Pixels':
 df_smoke_stop_test=pd.DataFrame({'Sample':[viper_id],
                                     'Test Date':[test_date],
                                    'Target Type':[target_type],
                                    'Ground Truth':[ground_truth],
                                    'Range Readout':[range_readout],
                                    'Range Accuracy':[range_accuracy],
                                    'Range Precision':[average_std],
                                    '90% PD Points':[pd_points],
                                    'Average Intensity':[average_int],
                                    'Min Intensity':[min_int],
                                    'Max Intensity':[max_int],
                                    'GT Target Size':[gt_target_size_area],
                                    'Target Area':[TargArea],
                                    'Mult Factor':[factor_area]
                                    })           
df_smoke_stop_test

In [ ]:
df_smoke_test_summary = pd.concat([df_smoke_t_lambertian, df_smoke_stop_test])

# DEFECTIVE PIXELS ANALYSIS

In [ ]:
%run $file_path_ingest
%run $file_path_calculations
sensor_type = "SRL" 

if os.path.exists(directory_pixels) != True :logger.warning(f"Directory not found!, Is the drive connected?")
else:
    sensor_viper = sensor(sensor_type, directory_pixels, debug=True)

viper_id, target_type, test_date = directory_data_extractor(directory_pixels)

In [ ]:
sensor_viper.data_read(multiple_files=False)
sensor_viper.row_col_fix()
sensor_viper.data_validation()

FAKE DEFECTIVE PIXELS GENERATION

In [ ]:
sensor_viper.df_valid_returns

In [ ]:
### Fake Defective Pixels Data Frame
df_copy = sensor_viper.df_valid_returns[sensor_viper.df_valid_returns['return_idx']==0].copy()
df_def_pix = df_copy[['intensity','col','row','frame_idx']]
condition = (df_def_pix['col'] == 21) & (df_def_pix['row'] == 32) | (df_def_pix['col'] == 45) & (df_def_pix['row'] == 32) | (df_def_pix['col'] == 100) & (df_def_pix['row'] == 31) | (df_def_pix['col'] == 101) & (df_def_pix['row'] == 31) | (df_def_pix['col'] == 100) & (df_def_pix['row'] == 25) 
df_def_pix.loc[condition, 'intensity'] = 0
# Identify which frames contain a given condition and then set the value of a column to a given value
# frame_idx_to_mod = df_def_pix.loc[(df_def_pix['row'] == 0) & (df_def_pix['col'] == 1), 'frame_idx'].unique()
# df_def_pix.loc[df_def_pix['frame_idx'].isin(frame_idx_to_mod), 'intensity']=0
px.scatter(df_def_pix,x='col',y='row',color='intensity',animation_frame='frame_idx')

FAKE DATA ANALYSIS

In [ ]:
# if target_type == 'Pixels':
#     if df_def_pix['intensity'].min()<10:
#         print("Evidence for Defective pixels found, minimum intensity found: ",df_def_pix['intensity'].min())
#         def_pix = df_def_pix[df_def_pix['intensity'] == 0]
#         def_pix['SAMPLE'] = viper_id
#         def_pix_summary = def_pix.groupby(['col', 'row','intensity'])['SAMPLE'].describe()
#         def_pix_summary.to_csv(f"{directory_pixels}/{viper_id}_Smoke_Test_Defective_Pixels_{get_current_time_date()}.csv", sep=',')
#         num_def_pix=def_pix['col'].count()/100
#         print('Number of Defective Pixels detected',num_def_pix)
#         df_smoke_test_summary['Def. Pixels'] = num_def_pix
#     else:
#         print("No evidence of dead pixels found, min int: ",df_def_pix['intensity'].min())

REAL DATA TESTING

In [ ]:
if target_type == 'Pixels':
    df_analysis_pix = sensor_viper.df_valid_returns[sensor_viper.df_valid_returns['return_idx']==0]
    if df_analysis_pix['intensity'].min()<10:
        print("Evidence for Defective pixels found, minimum intensity found: ",df_analysis_pix['intensity'].min())
        df_analysis_pix = df_analysis_pix[df_analysis_pix['intensity'] == 0]
        df_analysis_pix['SAMPLE'] = viper_id
        def_pix_summary = df_analysis_pix.groupby(['col', 'row','intensity'])[['SAMPLE']].describe()
        def_pix_summary.to_csv(f"{directory_pixels}/{viper_id}_Smoke_Test_Defective_Pixels_{get_current_time_date()}.csv", sep=',')
        num_def_pix = df_analysis_pix['col'].count()/100
        print('Number of Defective Pixels detected',num_def_pix)
        df_smoke_test_summary['Def. Pixels'] = num_def_pix
    else:
        print("No evidence of dead pixels found, min int: ",df_analysis_pix['intensity'].min())
        num_def_pix=0
        df_smoke_test_summary['Def. Pixels'] = num_def_pix

# SMOKE TEST RESULTS SUMMARY

In [ ]:
df_smoke_test_summary

CVS EXPORT OF RESULTS

In [ ]:
df_smoke_test_summary.to_csv(f"{directory_general}/{viper_id}_Smoke_Test_Summary_{get_current_time_date()}.csv", sep=',')